In [1]:
import pandas as pd
from datetime import tzinfo, timedelta, datetime
import numpy as np
import tarfile
from io import StringIO
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import itertools
from collections import Counter
import matplotlib.pylab as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import operator

In [2]:
def get_datetime_str(unixtime):
    fmt_datetime=datetime.fromtimestamp(float(str(unixtime))).strftime('%Y-%m-%d %H:%M:%S')
    fmt_timezone = "+00:00"
    return fmt_datetime + fmt_timezone

print get_datetime_str(1529076383.0)


2018-06-15 08:26:23+00:00


In [3]:
node_2o = pd.read_csv("/home/users/tleibengood/Documents/2018Poject/data/mine/node_2.csv")
node_5o = pd.read_csv("/home/users/tleibengood/Documents/2018Poject/data/mine/node_5.csv")
node_6o = pd.read_csv("/home/users/tleibengood/Documents/2018Poject/data/mine/node_6.csv")
node_7o = pd.read_csv("/home/users/tleibengood/Documents/2018Poject/data/mine/node_7.csv")
node_8o = pd.read_csv("/home/users/tleibengood/Documents/2018Poject/data/mine/node_8.csv")

In [4]:
print node_2o.shape
print node_5o.shape
print node_6o.shape
print node_7o.shape
print node_8o.shape

(3952817, 105)
(363488, 105)
(506512, 105)
(480739, 105)
(456868, 105)


In [5]:
3952817+363488+506512+480739+456868

5760424

In [61]:
c_columns =   ('c_port:2', 'c_pkts_all:3', 'c_rst_cnt:4', 'c_ack_cnt:5', 'c_ack_cnt_p:6', 'c_bytes_uniq:7', 'c_pkts_data:8', 
               'c_bytes_all:9', 'c_pkts_retx:10', 'c_bytes_retx:11', 'c_pkts_ooo:12', 'c_syn_cnt:13', 'c_fin_cnt:14',
               'durat:31', 'con_t:42', 'p2p_t:43', 'c_rtt_avg:45', 'c_rtt_min:46', 'c_rtt_max:47', 'c_rtt_std:48', 'c_rtt_cnt:49', 
               'c_ttl_min:50', 'c_ttl_max:51', 'c_f1323_opt:59', 'c_tm_opt:60', 'c_win_scl:61', 'c_sack_opt:62', 'c_sack_cnt:63', 
               'c_mss:64', 'c_mss_max:65', 'c_mss_min:66', 'c_win_max:67', 'c_win_min:68', 'c_win_0:69', 'c_cwin_max:70', 
               'c_cwin_min:71', 'c_cwin_ini:72', 'c_pkts_rto:73', 'c_pkts_fs:74', 'c_pkts_reor:75', 'c_pkts_dup:76', 'c_pkts_unk:77', 
               'c_pkts_fc:78', 'c_pkts_unrto:79', 'c_pkts_unfs:80', 'c_syn_retx:81')
s_columns =   ('s_port:16', 's_pkts_all:17', 's_rst_cnt:18', 's_ack_cnt:19', 's_ack_cnt_p:20', 's_bytes_uniq:21','s_pkts_data:22', 
               's_bytes_all:23', 's_pkts_retx:24', 's_bytes_retx:25', 's_pkts_ooo:26','s_syn_cnt:27', 's_fin_cnt:28', 
               'durat:31', 'con_t:42', 'p2p_t:43', 's_rtt_avg:52', 's_rtt_min:53', 's_rtt_max:54', 's_rtt_std:55', 's_rtt_cnt:56', 
               's_ttl_min:57', 's_ttl_max:58', 's_f1323_opt:82', 's_tm_opt:83', 's_win_scl:84', 's_sack_opt:85', 's_sack_cnt:86', 
               's_mss:87', 's_mss_max:88', 's_mss_min:89', 's_win_max:90', 's_win_min:91', 's_win_0:92', 's_cwin_max:93', 
               's_cwin_min:94', 's_cwin_ini:95', 's_pkts_rto:96', 's_pkts_fs:97', 's_pkts_reor:98', 's_pkts_dup:99', 's_pkts_unk:100', 
               's_pkts_fc:101', 's_pkts_unrto:102', 's_pkts_unfs:103', 's_syn_retx:104')
c_col_irrel = ('#09#c_ip:1', 'c_port:2', 'c_pkts_ooo:12', 'first:29', 'last:30', 'c_first:32', 'c_last:34', 'c_first_ack:36', 'c_isint:38', 
               'c_iscrypto:40', 'http_t:44', 'c_ttl_min:50', 'c_ttl_max:51', 'c_f1323_opt:59', 'c_tm_opt:60', 'c_win_scl:61', 
               'c_sack_opt:62', 'c_mss:64', 'c_pkts_fs:74', 'c_pkts_reor:75', 'c_pkts_dup:76', 'c_pkts_unk:77', 'c_pkts_fc:78', 
               'c_pkts_unfs:80', 'c_syn_retx:81', 
               's_ip:15', 's_port:16', 's_pkts_all:17', 's_rst_cnt:18', 's_ack_cnt:19', 's_ack_cnt_p:20', 's_bytes_uniq:21', 's_pkts_data:22', 
               's_bytes_all:23', 's_pkts_retx:24', 's_bytes_retx:25', 's_pkts_ooo:26','s_syn_cnt:27', 's_fin_cnt:28', 's_first:33', 
               's_last:35', 's_first_ack:37', 's_isint:39', 's_iscrypto:41', 's_rtt_avg:52', 's_rtt_min:53', 's_rtt_max:54', 's_rtt_std:55',
               's_rtt_cnt:56', 's_ttl_min:57', 's_ttl_max:58', 's_f1323_opt:82', 's_tm_opt:83', 's_win_scl:84', 's_sack_opt:85', 's_sack_cnt:86', 
               's_mss:87', 's_mss_max:88', 's_mss_min:89', 's_win_max:90', 's_win_min:91', 's_win_0:92', 's_cwin_max:93', 
               's_cwin_min:94', 's_cwin_ini:95', 's_pkts_rto:96', 's_pkts_fs:97', 's_pkts_reor:98', 's_pkts_dup:99', 's_pkts_unk:100', 
               's_pkts_fc:101', 's_pkts_unrto:102', 's_pkts_unfs:103', 's_syn_retx:104', )
s_col_irrel = ('s_ip:15', 's_port:16', 'first:29', 'last:30', 's_first:33', 's_last:35', 's_first_ack:37', 's_isint:39', 
               's_iscrypto:41', 'http_t:44', 's_ttl_min:57', 's_ttl_max:58', 's_f1323_opt:82', 's_tm_opt:83', 's_win_scl:84', 
               's_sack_opt:85', 's_mss:87', 's_win_min:91', 's_win_0:92', 's_pkts_fc:101', 's_syn_retx:104', 
               '#09#c_ip:1', 'c_port:2', 'c_pkts_all:3', 'c_rst_cnt:4', 'c_ack_cnt:5', 'c_ack_cnt_p:6', 'c_bytes_uniq:7', 'c_pkts_data:8', 
               'c_bytes_all:9', 'c_pkts_retx:10', 'c_bytes_retx:11', 'c_pkts_ooo:12', 'c_syn_cnt:13', 'c_fin_cnt:14', 'c_first:32', 
               'c_last:34', 'c_first_ack:36', 'c_isint:38', 'c_iscrypto:40', 'c_rtt_avg:45', 'c_rtt_min:46', 'c_rtt_max:47', 'c_rtt_std:48', 
               'c_rtt_cnt:49', 'c_ttl_min:50', 'c_ttl_max:51', 'c_f1323_opt:59', 'c_tm_opt:60', 'c_win_scl:61', 'c_sack_opt:62', 'c_sack_cnt:63', 
               'c_mss:64', 'c_mss_max:65', 'c_mss_min:66', 'c_win_max:67', 'c_win_min:68', 'c_win_0:69', 'c_cwin_max:70', 
               'c_cwin_min:71', 'c_cwin_ini:72', 'c_pkts_rto:73', 'c_pkts_fs:74', 'c_pkts_reor:75', 'c_pkts_dup:76', 'c_pkts_unk:77', 
               'c_pkts_fc:78', 'c_pkts_unrto:79', 'c_pkts_unfs:80', 'c_syn_retx:81')
irrel       = ('#09#c_ip:1', 'c_port:2', 'c_pkts_ooo:12', 'first:29', 'last:30', 'c_first:32', 'c_last:34', 'c_first_ack:36', 'c_isint:38', 
               'c_iscrypto:40', 'http_t:44', 'c_ttl_min:50', 'c_ttl_max:51', 'c_f1323_opt:59', 'c_tm_opt:60', 'c_win_scl:61', 
               'c_sack_opt:62', 'c_mss:64', 'c_pkts_fs:74', 'c_pkts_reor:75', 'c_pkts_dup:76', 'c_pkts_unk:77', 'c_pkts_fc:78', 
               'c_pkts_unfs:80', 'c_syn_retx:81', 's_ip:15', 's_port:16', 'first:29', 'last:30', 's_first:33', 's_last:35', 's_first_ack:37', 's_isint:39', 
               's_iscrypto:41', 'http_t:44', 's_ttl_min:57', 's_ttl_max:58', 's_f1323_opt:82', 's_tm_opt:83', 's_win_scl:84', 
               's_sack_opt:85', 's_mss:87', 's_win_min:91', 's_win_0:92', 's_pkts_fc:101', 's_syn_retx:104')
ports       = ('c_port:2', 's_port:16', '#09#c_ip:1', 's_ip:15', 'first:29', 'last:30', 'c_first:32', 'c_last:34', 'c_first_ack:36', 'c_isint:38', 'c_iscrypto:40', 's_first:33', 
               's_last:35', 's_first_ack:37', 's_isint:39', 's_iscrypto:41', 'http_t:44')
non_sca     = ('#09#c_ip:1', 's_ip:15', 'first:29', 'last:30', 'c_first:32', 'c_last:34', 'c_first_ack:36', 'c_isint:38', 'c_iscrypto:40', 's_first:33', 
               's_last:35', 's_first_ack:37', 's_isint:39', 's_iscrypto:41', 'http_t:44')
Mariam      = ['c_rtt_avg:45', 's_rtt_avg:52','c_mss_max:65','c_mss_min:66','c_win_max:67','c_win_min:68','c_win_0:69',
               'c_cwin_max:70','c_cwin_min:71','c_cwin_ini:72','c_pkts_rto:73','c_pkts_fs:74','c_pkts_reor:75','c_pkts_dup:76',
               's_mss_max:88','s_mss_min:89','s_win_max:90','s_win_min:91','s_win_0:92','s_cwin_max:93','s_cwin_min:94',
               's_cwin_ini:95','s_pkts_rto:96','s_pkts_fs:97','s_pkts_reor:98','s_pkts_dup:99']
Mariam_c    = ['c_rtt_avg:45','c_mss_max:65','c_mss_min:66','c_win_max:67','c_win_min:68','c_win_0:69','c_cwin_max:70',
               'c_cwin_min:71','c_cwin_ini:72','c_pkts_rto:73','c_pkts_fs:74','c_pkts_reor:75','c_pkts_dup:76',]
Mariam_s    = ['s_rtt_avg:52','s_mss_max:88','s_mss_min:89','s_win_max:90','s_win_min:91','s_win_0:92','s_cwin_max:93',
               's_cwin_min:94','s_cwin_ini:95','s_pkts_rto:96','s_pkts_fs:97','s_pkts_reor:98','s_pkts_dup:99']

In [62]:
node_2o = node_2o[(node_2o['first:29'] > 1493596800000.0) &
                  (node_2o['first:29'] < 1497139200000.0)]
node_5o = node_5o[(node_5o['first:29'] > 1493596800000.0) &
                  (node_5o['first:29'] < 1497139200000.0)]
node_6o = node_6o[(node_6o['first:29'] > 1493596800000.0) &
                  (node_6o['first:29'] < 1497139200000.0)]
node_7o = node_7o[(node_7o['first:29'] > 1493596800000.0) &
                  (node_7o['first:29'] < 1497139200000.0)]
node_8o = node_8o[(node_8o['first:29'] > 1493596800000.0) &
                  (node_8o['first:29'] < 1497139200000.0)]

In [63]:
c_node_2 = node_2o[list(c_columns)]
s_node_2 = node_2o[list(s_columns)]
c_node_5 = node_5o[list(c_columns)]
s_node_5 = node_5o[list(s_columns)]
c_node_6 = node_6o[list(c_columns)]
s_node_6 = node_6o[list(s_columns)]
c_node_7 = node_7o[list(c_columns)]
s_node_7 = node_7o[list(s_columns)]
c_node_8 = node_8o[list(c_columns)]
s_node_8 = node_8o[list(s_columns)]

c_node_2_rel = node_2o.drop(columns = list(c_col_irrel))
s_node_2_rel = node_2o.drop(columns = list(s_col_irrel))
c_node_5_rel = node_5o.drop(columns = list(c_col_irrel))
s_node_5_rel = node_5o.drop(columns = list(s_col_irrel))
c_node_6_rel = node_6o.drop(columns = list(c_col_irrel))
s_node_6_rel = node_6o.drop(columns = list(s_col_irrel))
c_node_7_rel = node_7o.drop(columns = list(c_col_irrel))
s_node_7_rel = node_7o.drop(columns = list(s_col_irrel))
c_node_8_rel = node_8o.drop(columns = list(c_col_irrel))
s_node_8_rel = node_8o.drop(columns = list(s_col_irrel))

node_2 = node_2o.drop(columns = list(non_sca))
node_5 = node_5o.drop(columns = list(non_sca))
node_6 = node_6o.drop(columns = list(non_sca))
node_7 = node_7o.drop(columns = list(non_sca))
node_8 = node_8o.drop(columns = list(non_sca))

node_2_rel = node_2o.drop(columns = list(irrel))
node_5_rel = node_5o.drop(columns = list(irrel))
node_6_rel = node_6o.drop(columns = list(irrel))
node_7_rel = node_7o.drop(columns = list(irrel))
node_8_rel = node_8o.drop(columns = list(irrel))

no_ports_2 = node_2o.drop(columns = list(ports))
no_ports_5 = node_5o.drop(columns = list(ports))
no_ports_6 = node_6o.drop(columns = list(ports))
no_ports_7 = node_7o.drop(columns = list(ports))
no_ports_8 = node_8o.drop(columns = list(ports))

Mariam_2 = node_2o[Mariam]
Mariam_5 = node_5o[Mariam]
Mariam_6 = node_6o[Mariam]
Mariam_7 = node_7o[Mariam]
Mariam_8 = node_8o[Mariam]

Mariam_s_2 = node_2o[Mariam_s]
Mariam_s_5 = node_5o[Mariam_s]
Mariam_s_6 = node_6o[Mariam_s]
Mariam_s_7 = node_7o[Mariam_s]
Mariam_s_8 = node_8o[Mariam_s]

Mariam_c_2 = node_2o[Mariam_c]
Mariam_c_5 = node_5o[Mariam_c]
Mariam_c_6 = node_6o[Mariam_c]
Mariam_c_7 = node_7o[Mariam_c]
Mariam_c_8 = node_8o[Mariam_c]

In [64]:
c_node_2_rel = c_node_2_rel.drop(columns = c_node_2_rel.columns.values[0])
s_node_2_rel = s_node_2_rel.drop(columns = s_node_2_rel.columns.values[0])
c_node_5_rel = c_node_5_rel.drop(columns = c_node_5_rel.columns.values[0])
s_node_5_rel = s_node_5_rel.drop(columns = s_node_5_rel.columns.values[0])
c_node_6_rel = c_node_6_rel.drop(columns = c_node_6_rel.columns.values[0])
s_node_6_rel = s_node_6_rel.drop(columns = s_node_6_rel.columns.values[0])
c_node_7_rel = c_node_7_rel.drop(columns = c_node_7_rel.columns.values[0])
s_node_7_rel = s_node_7_rel.drop(columns = s_node_7_rel.columns.values[0])
c_node_8_rel = c_node_8_rel.drop(columns = c_node_8_rel.columns.values[0])
s_node_8_rel = s_node_8_rel.drop(columns = s_node_8_rel.columns.values[0])

node_2 = node_2.drop(columns = node_2.columns.values[0])
node_5 = node_5.drop(columns = node_5.columns.values[0])
node_6 = node_6.drop(columns = node_6.columns.values[0])
node_7 = node_7.drop(columns = node_7.columns.values[0])
node_8 = node_8.drop(columns = node_8.columns.values[0])

node_2_rel = node_2_rel.drop(columns = node_2_rel.columns.values[0])
node_5_rel = node_5_rel.drop(columns = node_5_rel.columns.values[0])
node_6_rel = node_6_rel.drop(columns = node_6_rel.columns.values[0])
node_7_rel = node_7_rel.drop(columns = node_7_rel.columns.values[0])
node_8_rel = node_8_rel.drop(columns = node_8_rel.columns.values[0])

no_ports_2 = no_ports_2.drop(columns = no_ports_2.columns.values[0])
no_ports_5 = no_ports_5.drop(columns = no_ports_5.columns.values[0])
no_ports_6 = no_ports_6.drop(columns = no_ports_6.columns.values[0])
no_ports_7 = no_ports_7.drop(columns = no_ports_7.columns.values[0])
no_ports_8 = no_ports_8.drop(columns = no_ports_8.columns.values[0])

In [65]:
Fc_node_2 = list(c_node_2.columns.values)
Fs_node_2 = list(s_node_2.columns.values)
Fc_node_5 = list(c_node_5.columns.values)
Fs_node_5 = list(s_node_5.columns.values)
Fc_node_6 = list(c_node_6.columns.values)
Fs_node_6 = list(s_node_6.columns.values)
Fc_node_7 = list(c_node_7.columns.values)
Fs_node_7 = list(s_node_7.columns.values)
Fc_node_8 = list(c_node_8.columns.values)
Fs_node_8 = list(s_node_8.columns.values)

Fc_node_2_rel = list(c_node_2_rel.columns.values)
Fs_node_2_rel = list(s_node_2_rel.columns.values)
Fc_node_5_rel = list(c_node_5_rel.columns.values)
Fs_node_5_rel = list(s_node_5_rel.columns.values)
Fc_node_6_rel = list(c_node_6_rel.columns.values)
Fs_node_6_rel = list(s_node_6_rel.columns.values)
Fc_node_7_rel = list(c_node_7_rel.columns.values)
Fs_node_7_rel = list(s_node_7_rel.columns.values)
Fc_node_8_rel = list(c_node_8_rel.columns.values)
Fs_node_8_rel = list(s_node_8_rel.columns.values)

Fnode_2 = list(node_2.columns.values)
Fnode_5 = list(node_5.columns.values)
Fnode_6 = list(node_6.columns.values)
Fnode_7 = list(node_7.columns.values)
Fnode_8 = list(node_8.columns.values)

Fnode_2_rel = list(node_2_rel.columns.values)
Fnode_5_rel = list(node_5_rel.columns.values)
Fnode_6_rel = list(node_6_rel.columns.values)
Fnode_7_rel = list(node_7_rel.columns.values)
Fnode_8_rel = list(node_8_rel.columns.values)

Fno_ports_2 = list(no_ports_2.columns.values)
Fno_ports_5 = list(no_ports_5.columns.values)
Fno_ports_6 = list(no_ports_6.columns.values)
Fno_ports_7 = list(no_ports_7.columns.values)
Fno_ports_8 = list(no_ports_8.columns.values)

FMariam_2 = list(Mariam_2.columns.values)
FMariam_5 = list(Mariam_5.columns.values)
FMariam_6 = list(Mariam_6.columns.values)
FMariam_7 = list(Mariam_7.columns.values)
FMariam_8 = list(Mariam_8.columns.values)

FMariam_s_2 = list(Mariam_s_2.columns.values)
FMariam_s_5 = list(Mariam_s_5.columns.values)
FMariam_s_6 = list(Mariam_s_6.columns.values)
FMariam_s_7 = list(Mariam_s_7.columns.values)
FMariam_s_8 = list(Mariam_s_8.columns.values)

FMariam_c_2 = list(Mariam_c_2.columns.values)
FMariam_c_5 = list(Mariam_c_5.columns.values)
FMariam_c_6 = list(Mariam_c_6.columns.values)
FMariam_c_7 = list(Mariam_c_7.columns.values)
FMariam_c_8 = list(Mariam_c_8.columns.values)

In [66]:
print Fc_node_2[0]
print Fs_node_2[0]
print Fc_node_5[0]
print Fs_node_5[0]
print Fc_node_6[0]
print Fs_node_6[0]
print Fc_node_7[0]
print Fs_node_7[0]
print Fc_node_8[0]
print Fs_node_8[0]
print 
print Fc_node_2_rel[0]
print Fs_node_2_rel[0]
print Fc_node_5_rel[0]
print Fs_node_5_rel[0]
print Fc_node_6_rel[0]
print Fs_node_6_rel[0]
print Fc_node_7_rel[0]
print Fs_node_7_rel[0]
print Fc_node_8_rel[0]
print Fs_node_8_rel[0]
print 
print Fnode_2[0]
print Fnode_5[0]
print Fnode_6[0]
print Fnode_7[0]
print Fnode_8[0]
print 
print Fnode_2_rel[0]
print Fnode_5_rel[0]
print Fnode_6_rel[0]
print Fnode_7_rel[0]
print Fnode_8_rel[0]
print
print Fno_ports_2[0]
print Fno_ports_5[0]
print Fno_ports_6[0]
print Fno_ports_7[0]
print Fno_ports_8[0]

c_port:2
s_port:16
c_port:2
s_port:16
c_port:2
s_port:16
c_port:2
s_port:16
c_port:2
s_port:16

c_pkts_all:3
s_pkts_all:17
c_pkts_all:3
s_pkts_all:17
c_pkts_all:3
s_pkts_all:17
c_pkts_all:3
s_pkts_all:17
c_pkts_all:3
s_pkts_all:17

c_port:2
c_port:2
c_port:2
c_port:2
c_port:2

c_pkts_all:3
c_pkts_all:3
c_pkts_all:3
c_pkts_all:3
c_pkts_all:3

c_pkts_all:3
c_pkts_all:3
c_pkts_all:3
c_pkts_all:3
c_pkts_all:3


In [67]:
print c_node_2.shape
print c_node_5.shape
print c_node_6.shape
print c_node_7.shape
print c_node_8.shape
print
print c_node_2_rel.shape
print c_node_5_rel.shape
print c_node_6_rel.shape
print c_node_7_rel.shape
print c_node_8_rel.shape
print
print s_node_2.shape
print s_node_5.shape
print s_node_6.shape
print s_node_7.shape
print s_node_8.shape
print
print s_node_2_rel.shape
print s_node_5_rel.shape
print s_node_6_rel.shape
print s_node_7_rel.shape
print s_node_8_rel.shape

(738460, 46)
(222125, 46)
(335671, 46)
(271495, 46)
(312302, 46)

(738460, 30)
(222125, 30)
(335671, 30)
(271495, 30)
(312302, 30)

(738460, 46)
(222125, 46)
(335671, 46)
(271495, 46)
(312302, 46)

(738460, 34)
(222125, 34)
(335671, 34)
(271495, 34)
(312302, 34)


In [68]:
node_2 = pd.DataFrame(MinMaxScaler().fit_transform(node_2))
node_5 = pd.DataFrame(MinMaxScaler().fit_transform(node_5))
node_6 = pd.DataFrame(MinMaxScaler().fit_transform(node_6))
node_7 = pd.DataFrame(MinMaxScaler().fit_transform(node_7))
node_8 = pd.DataFrame(MinMaxScaler().fit_transform(node_8))

node_2_rel = pd.DataFrame(MinMaxScaler().fit_transform(node_2_rel))
node_5_rel = pd.DataFrame(MinMaxScaler().fit_transform(node_5_rel))
node_6_rel = pd.DataFrame(MinMaxScaler().fit_transform(node_6_rel))
node_7_rel = pd.DataFrame(MinMaxScaler().fit_transform(node_7_rel))
node_8_rel = pd.DataFrame(MinMaxScaler().fit_transform(node_8_rel))

c_node_2_rel = pd.DataFrame(MinMaxScaler().fit_transform(c_node_2_rel))
s_node_2_rel = pd.DataFrame(MinMaxScaler().fit_transform(c_node_2_rel))
c_node_5_rel = pd.DataFrame(MinMaxScaler().fit_transform(c_node_5_rel))
s_node_5_rel = pd.DataFrame(MinMaxScaler().fit_transform(c_node_5_rel))
c_node_6_rel = pd.DataFrame(MinMaxScaler().fit_transform(c_node_6_rel))
s_node_6_rel = pd.DataFrame(MinMaxScaler().fit_transform(c_node_6_rel))
c_node_7_rel = pd.DataFrame(MinMaxScaler().fit_transform(c_node_7_rel))
s_node_7_rel = pd.DataFrame(MinMaxScaler().fit_transform(c_node_7_rel))
c_node_8_rel = pd.DataFrame(MinMaxScaler().fit_transform(c_node_8_rel))
s_node_8_rel = pd.DataFrame(MinMaxScaler().fit_transform(c_node_8_rel))

c_node_2 = pd.DataFrame(MinMaxScaler().fit_transform(c_node_2))
c_node_5 = pd.DataFrame(MinMaxScaler().fit_transform(c_node_5))
c_node_6 = pd.DataFrame(MinMaxScaler().fit_transform(c_node_6))
c_node_7 = pd.DataFrame(MinMaxScaler().fit_transform(c_node_7))
c_node_8 = pd.DataFrame(MinMaxScaler().fit_transform(c_node_8))
s_node_2 = pd.DataFrame(MinMaxScaler().fit_transform(s_node_2))
s_node_5 = pd.DataFrame(MinMaxScaler().fit_transform(s_node_5))
s_node_6 = pd.DataFrame(MinMaxScaler().fit_transform(s_node_6))
s_node_7 = pd.DataFrame(MinMaxScaler().fit_transform(s_node_7))
s_node_8 = pd.DataFrame(MinMaxScaler().fit_transform(s_node_8))

no_ports_2 = pd.DataFrame(MinMaxScaler().fit_transform(no_ports_2))
no_ports_5 = pd.DataFrame(MinMaxScaler().fit_transform(no_ports_5))
no_ports_6 = pd.DataFrame(MinMaxScaler().fit_transform(no_ports_6))
no_ports_7 = pd.DataFrame(MinMaxScaler().fit_transform(no_ports_7))
no_ports_8 = pd.DataFrame(MinMaxScaler().fit_transform(no_ports_8))

Mariam_2 = pd.DataFrame(MinMaxScaler().fit_transform(Mariam_2))
Mariam_5 = pd.DataFrame(MinMaxScaler().fit_transform(Mariam_5))
Mariam_6 = pd.DataFrame(MinMaxScaler().fit_transform(Mariam_6))
Mariam_7 = pd.DataFrame(MinMaxScaler().fit_transform(Mariam_7))
Mariam_8 = pd.DataFrame(MinMaxScaler().fit_transform(Mariam_8))

Mariam_s_2 = pd.DataFrame(MinMaxScaler().fit_transform(Mariam_s_2))
Mariam_s_5 = pd.DataFrame(MinMaxScaler().fit_transform(Mariam_s_5))
Mariam_s_6 = pd.DataFrame(MinMaxScaler().fit_transform(Mariam_s_6))
Mariam_s_7 = pd.DataFrame(MinMaxScaler().fit_transform(Mariam_s_7))
Mariam_s_8 = pd.DataFrame(MinMaxScaler().fit_transform(Mariam_s_8))

Mariam_c_2 = pd.DataFrame(MinMaxScaler().fit_transform(Mariam_c_2))
Mariam_c_5 = pd.DataFrame(MinMaxScaler().fit_transform(Mariam_c_5))
Mariam_c_6 = pd.DataFrame(MinMaxScaler().fit_transform(Mariam_c_6))
Mariam_c_7 = pd.DataFrame(MinMaxScaler().fit_transform(Mariam_c_7))
Mariam_c_8 = pd.DataFrame(MinMaxScaler().fit_transform(Mariam_c_8))

In [113]:
def sub_comp(X, y):
    pca = PCA(n_components=1)
    pca.fit(X)
    print pca.explained_variance_ratio_
    comp_01 = abs(pca.components_[0])
    return sorted(zip(comp_01, y), reverse=True)[:2]

In [26]:
sc_node_2 = sub_comp(node_2, Fnode_2)
sc_node_5 = sub_comp(node_5, Fnode_5)
sc_node_6 = sub_comp(node_6, Fnode_6)
sc_node_7 = sub_comp(node_7, Fnode_7)
sc_node_8 = sub_comp(node_8, Fnode_8)

[0.38144163]
[0.4620557]
[0.40543341]
[0.47383956]
[0.40435692]


In [27]:
sc_node_2_rel = sub_comp(node_2_rel, Fnode_2_rel)
sc_node_5_rel = sub_comp(node_5_rel, Fnode_5_rel)
sc_node_6_rel = sub_comp(node_6_rel, Fnode_6_rel)
sc_node_7_rel = sub_comp(node_7_rel, Fnode_7_rel)
sc_node_8_rel = sub_comp(node_8_rel, Fnode_8_rel)

[0.38567554]
[0.42661641]
[0.44194138]
[0.49267943]
[0.42438929]


In [28]:
sc_c_node_2 = sub_comp(c_node_2, Fc_node_2)
sc_c_node_5 = sub_comp(c_node_5, Fc_node_5)
sc_c_node_6 = sub_comp(c_node_6, Fc_node_6)
sc_c_node_7 = sub_comp(c_node_7, Fc_node_7)
sc_c_node_8 = sub_comp(c_node_8, Fc_node_8)

sc_s_node_2 = sub_comp(s_node_2, Fs_node_2)
sc_s_node_5 = sub_comp(s_node_5, Fs_node_5)
sc_s_node_6 = sub_comp(s_node_6, Fs_node_6)
sc_s_node_7 = sub_comp(s_node_7, Fs_node_7)
sc_s_node_8 = sub_comp(s_node_8, Fs_node_8)

[0.42294503]
[0.58835441]
[0.51726183]
[0.58535512]
[0.51857321]
[0.50276637]
[0.54704954]
[0.52857755]
[0.57032967]
[0.56430349]


In [29]:
sc_c_node_2_rel = sub_comp(c_node_2_rel, Fc_node_2_rel)
sc_c_node_5_rel = sub_comp(c_node_5_rel, Fc_node_5_rel)
sc_c_node_6_rel = sub_comp(c_node_6_rel, Fc_node_6_rel)
sc_c_node_7_rel = sub_comp(c_node_7_rel, Fc_node_7_rel)
sc_c_node_8_rel = sub_comp(c_node_8_rel, Fc_node_8_rel)

sc_s_node_2_rel = sub_comp(s_node_2_rel, Fs_node_2_rel)
sc_s_node_5_rel = sub_comp(s_node_5_rel, Fs_node_5_rel)
sc_s_node_6_rel = sub_comp(s_node_6_rel, Fs_node_6_rel)
sc_s_node_7_rel = sub_comp(s_node_7_rel, Fs_node_7_rel)
sc_s_node_8_rel = sub_comp(s_node_8_rel, Fs_node_8_rel)

[0.5389457]
[0.59421761]
[0.69334001]
[0.688436]
[0.57495725]
[0.5389457]
[0.59421761]
[0.69334001]
[0.688436]
[0.57495725]


In [30]:
sc_no_ports_2 = sub_comp(no_ports_2, Fno_ports_2)
sc_no_ports_5 = sub_comp(no_ports_5, Fno_ports_5)
sc_no_ports_6 = sub_comp(no_ports_6, Fno_ports_6)
sc_no_ports_7 = sub_comp(no_ports_7, Fno_ports_7)
sc_no_ports_8 = sub_comp(no_ports_8, Fno_ports_8)

[0.43949972]
[0.40003462]
[0.33382552]
[0.38972407]
[0.3472257]


In [114]:
sc_Mariam_2 = sub_comp(Mariam_2, FMariam_2)
sc_Mariam_5 = sub_comp(Mariam_5, FMariam_5)
sc_Mariam_6 = sub_comp(Mariam_6, FMariam_6)
sc_Mariam_7 = sub_comp(Mariam_7, FMariam_7)
sc_Mariam_8 = sub_comp(Mariam_8, FMariam_8)

sc_Mariam_s_2 = sub_comp(Mariam_s_2, FMariam_s_2)
sc_Mariam_s_5 = sub_comp(Mariam_s_5, FMariam_s_5)
sc_Mariam_s_6 = sub_comp(Mariam_s_6, FMariam_s_6)
sc_Mariam_s_7 = sub_comp(Mariam_s_7, FMariam_s_7)
sc_Mariam_s_8 = sub_comp(Mariam_s_8, FMariam_s_8)

sc_Mariam_c_2 = sub_comp(Mariam_c_2, FMariam_c_2)
sc_Mariam_c_5 = sub_comp(Mariam_c_5, FMariam_c_5)
sc_Mariam_c_6 = sub_comp(Mariam_c_6, FMariam_c_6)
sc_Mariam_c_7 = sub_comp(Mariam_c_7, FMariam_c_7)
sc_Mariam_c_8 = sub_comp(Mariam_c_8, FMariam_c_8)

[0.5938505]
[0.61974393]
[0.55365022]
[0.67138688]
[0.60897001]
[0.46913367]
[0.40186505]
[0.40999206]
[0.48218905]
[0.42268554]
[0.80772191]
[0.86906748]
[0.88889371]
[0.90553934]
[0.86135087]


In [79]:
print sc_node_2
print sc_c_node_2
print sc_s_node_2
print sc_c_node_2_rel
print sc_s_node_2_rel
print sc_no_ports_2

[(0.7691146747481162, 's_mss:87'), (0.374691145629104, 'c_mss_max:65'), (0.2979219482729609, 'c_mss:64')]
[(0.7882108561035659, 'c_mss:64'), (0.4591800309395497, 'c_mss_max:65'), (0.3315474805335333, 'c_win_scl:61')]
[(0.8687466130309505, 's_mss:87'), (0.3231635400206383, 's_port:16'), (0.2539491180753881, 's_win_scl:84')]
[(0.9527854926509581, 'c_mss_max:65'), (0.19424552105368786, 'con_t:42'), (0.14681277549029073, 'c_cwin_max:70')]
[(0.9527854926509578, 's_sack_cnt:86'), (0.19424552105368786, 'durat:31'), (0.14681277549029068, 's_cwin_min:94')]
[(0.795304005002492, 's_mss:87'), (0.37370415025978587, 'c_mss:64'), (0.3257414648963244, 'c_mss_max:65')]


In [115]:
sc         = sc_node_2 + sc_node_5 + sc_node_6 + sc_node_7 + sc_node_8
sc_c       = sc_c_node_2 + sc_c_node_5 + sc_c_node_6 + sc_c_node_7 + sc_c_node_8
sc_s       = sc_s_node_2 + sc_s_node_5 + sc_s_node_6 + sc_s_node_7 + sc_s_node_8 
sc_rel     = sc_node_2_rel + sc_node_5_rel + sc_node_6_rel + sc_node_7_rel + sc_node_8_rel
sc_c_rel   = sc_c_node_2_rel + sc_c_node_5_rel + sc_c_node_6_rel + sc_c_node_7_rel + sc_c_node_8_rel
sc_s_rel   = sc_s_node_2_rel + sc_s_node_5_rel + sc_s_node_6_rel + sc_s_node_7_rel + sc_s_node_8_rel
sc_no_port = sc_no_ports_2 + sc_no_ports_5 + sc_no_ports_6 + sc_no_ports_7 + sc_no_ports_8
sc_Mariam     = sc_Mariam_2 + sc_Mariam_5 + sc_Mariam_6 + sc_Mariam_7 + sc_Mariam_8
sc_Mariam_s   = sc_Mariam_s_2 + sc_Mariam_s_5 + sc_Mariam_s_6 + sc_Mariam_s_7 + sc_Mariam_s_8
sc_Mariam_c   = sc_Mariam_c_2 + sc_Mariam_c_5 + sc_Mariam_c_6 + sc_Mariam_c_7 + sc_Mariam_c_8

In [72]:
def dom_sc(X) :
    lis = zip(*X)[1]
    counts = Counter(lis).items()
    counts.sort(key=operator.itemgetter(1), reverse = True)
    return counts[:10]

In [91]:
dom_sc(sc)

[(0.7691146747481001, 's_mss:87'), (0.3746911456290443, 'c_mss_max:65'), (0.297921948272904, 'c_mss:64'), (0.25504205580128314, 's_port:16'), (0.6379660020394812, 's_port:16'), (0.48993815838364474, 'c_mss:64'), (0.43989370861839394, 'c_mss_max:65'), (0.24682843443064031, 'c_win_scl:61'), (0.6271253745529493, 's_port:16'), (0.5512544905215495, 'c_mss_max:65'), (0.23487906719054752, 's_mss:87'), (0.23289408055740948, 'c_mss:64'), (0.6234310589981042, 's_port:16'), (0.5268639547808627, 'c_mss_max:65'), (0.3829382570148829, 'c_mss:64'), (0.20414577833554068, 's_mss:87'), (0.7071600675461047, 's_port:16'), (0.39890710934694834, 'c_mss_max:65'), (0.3583195665163158, 'c_mss:64'), (0.23564370893240047, 's_mss:87')]


[('c_mss_max:65', 5),
 ('s_port:16', 5),
 ('c_mss:64', 5),
 ('s_mss:87', 4),
 ('c_win_scl:61', 1)]

In [105]:
dom_sc(sc_c)

[('c_mss:64', 5),
 ('c_win_scl:61', 5),
 ('c_mss_max:65', 5),
 ('con_t:42', 3),
 ('c_cwin_max:70', 1),
 ('c_rst_cnt:4', 1)]

In [101]:
dom_sc(sc_s)

[('s_port:16', 5),
 ('s_mss:87', 5),
 ('s_rst_cnt:18', 3),
 ('s_win_scl:84', 1),
 ('s_mss_min:89', 1)]

In [97]:
dom_sc(sc_rel)

[('c_mss_max:65', 5),
 ('s_rst_cnt:18', 3),
 ('c_cwin_max:70', 2),
 ('s_syn_cnt:27', 2),
 ('s_mss_min:89', 2),
 ('con_t:42', 1)]

In [108]:
dom_sc(sc_c_rel)

[('c_cwin_max:70', 5),
 ('c_mss_max:65', 5),
 ('con_t:42', 5),
 ('c_win_min:68', 4),
 ('c_rst_cnt:4', 1)]

In [109]:
dom_sc(sc_s_rel)

[('s_cwin_min:94', 5),
 ('durat:31', 5),
 ('s_sack_cnt:86', 5),
 ('s_win_max:90', 4),
 ('s_rst_cnt:18', 1)]

In [121]:
dom_sc(sc_no_port)

[('c_mss:64', 4),
 ('c_mss_max:65', 4),
 ('s_mss:87', 3),
 ('c_win_scl:61', 2),
 ('s_rst_cnt:18', 2)]

In [111]:
dom_sc(sc_Mariam)

[('c_mss_max:65', 5),
 ('c_cwin_max:70', 3),
 ('s_mss_min:89', 3),
 ('c_win_min:68', 2),
 ('s_pkts_dup:99', 1),
 ('s_win_max:90', 1)]

In [116]:
dom_sc(sc_Mariam_s)

[('s_mss_min:89', 3),
 ('s_pkts_dup:99', 2),
 ('s_cwin_min:94', 2),
 ('s_mss_max:88', 1),
 ('s_win_min:91', 1),
 ('s_cwin_max:93', 1)]

In [105]:
dom_sc(sc_Mariam_c)

[('c_cwin_max:70', 5),
 ('c_mss_max:65', 5),
 ('c_win_min:68', 5),
 ('c_rtt_avg:45', 4),
 ('c_cwin_min:71', 1)]